#### **This notebook get the tweetids that were replied by the IO accounts, save it to text files and gets the replies to these tweetids (accounts outside) IO**

In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config

##### ***Load config***

In [2]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']

In [3]:
file_hp.create_folder(derived_path, 'conversation')

conversation_path = os.path.join(derived_path, 
                                 'conversation')

##### ***Getting replied_to_tweet ids for replies to external accounts***

In [4]:
def save_replied_to_tweetid_external(df, file_name,
                                     conversation_path,
                                    ):
    ''' 
    Gets in_reply_to_tweetid tweetids and save it to text file

    :param all_campaigns: dictionary of all campaigns
    :param all_tweet_data: path to all tweet data
    :param conversation_path: path to save the conversation/in_reply_to_tweetid ids
    '''

    if 'in_reply_to_tweetid' not in df.columns:
        return False

    df = st.reply_to_external_users(df, 'all')

    df = df.astype({
        'in_reply_to_tweetid': int
    })

    rows = df.loc[
        df['in_reply_to_tweetid'] != 0]['in_reply_to_tweetid'].unique()

    if len(rows) == 0:
        return False


    file_hp.write_to_file_row_each_line(conversation_path, 
                                        file_name,
                                        rows)
    return True

In [5]:
all_campaigns, names = st.bundle_campaign()

def run_for_all_campaign(all_campaigns, 
                         all_tweet_data,
                         conversation_path, 
                         type_of='ops'):
    '''
    Gets in_reply_to_tweetid tweetids and save it to text file 
    for all_campaigns
    
    :param all_campaigns: dictionary of all campaigns
    :param all_tweet_data: path to all tweet data
    :param conversation_path: path to save the conversation/in_reply_to_tweetid ids
    '''
    for row in all_campaigns:
        for year in row:
            for new_campaign in row[year]:

                print(f'\n ------START: {year}: {new_campaign} ------- \n')

                ops_file_path = os.path.join(all_tweet_data, year, 
                                             new_campaign, 
                                             f'{new_campaign}_tweets.pkl.gz')
                control_file_path = os.path.join(all_tweet_data, 
                                                 year, 
                                                 new_campaign, 
                                                 'DriversControl', 
                                 f'{new_campaign}_tweets_control.pkl.gz')

                data = st.read_ops_control_data(ops_file_path, 
                                             control_file_path, 
                                             [type_of])

                df = data[type_of]
                
                if len(df) == 0 and type_of == 'control':
                    control_file_path = os.path.join(all_tweet_data, 
                                                     year, 
                                                     new_campaign, 
                                                     'DriversControl', 
                                                     f'{new_campaign}_control.pkl.gz')
                    df = st.read_ops_control_data(ops_file_path, 
                                             control_file_path, 
                                             [type_of])['control']
                
                if len(df_test) == 0:
                    print(f'\n ------END: {year}: {new_campaign} ------- \n')
                    continue
                    
                file_name = f'{campaign}_{type_of}_conversation_ids.txt'
                
                flag = save_replied_to_tweetid_external(df,
                                                 conversation_path,
                                                 file_name)
                
                if flag == False:
                    print('Problem in extrating conversation ids')
                    
                    return
                
                print(f'\n ------END: {year}: {new_campaign} ------- \n')

In [5]:
# run_for_all_campaign(all_campaigns, all_tweet_data,
#                                     conversation_path)

##### ***Search conversation_ids***

In [6]:
def get_conversation(conversation_file,
                     extraction_location,
                     filename=None
                    ):
    '''
    Get the conversation from conversation ids (conversation ids
    are in txt file. Each line is one conversation id
    
    :param conversation_file: location where conversation file is
    located
    :param extraction_location: location where extracted conversation is
    saved
    '''
    for id_file in glob.glob(conversation_file):
        conversation_id_file = id_file.split(os.sep)[-1]
        campaign = conversation_id_file.split('.')[0]

        print(f'\n\n ---- Starting for campaign : {campaign} -- \n')

        if filename == None:
            campaign_json = f'{campaign}.jsonl'
        else:
            campaign_json = filename
            
        path_to_json = os.path.join(extraction_location, campaign_json)
        command = f'twarc2 conversations  --archive {id_file} > {path_to_json}'

        os.system(command)  

        print(f'\n ---- Ending for campaign : {campaign} ------- \n')

In [7]:
conversation_file = os.path.join(conversation_path, 
                                 'iran_202012_conversation_ids.txt')

file_hp.create_folder(derived_path,
                      'extracted_conversation')
extraction_location = os.path.join(derived_path, 
                                   'extracted_conversation')


get_conversation(conversation_file, extraction_location)



 ---- Starting for campaign : iran_202012_conversation_ids -- 


 ---- Ending for campaign : iran_202012_conversation_ids ------- 




Aborted!


In [ ]:
###Testing for control